Вот несколько стратегий, которые можно использовать для отбора наиболее значимых полиномиальных признаков с использованием CatBoost:

**1. Feature Importance из CatBoost:**  
Суть: CatBoost предоставляет встроенную возможность оценки важности признаков. После обучения модели можно извлечь эту информацию и использовать ее для отбора наиболее важных полиномиальных признаков.  
Преимущества: Простота реализации, встроенный в CatBoost.  
Недостатки: Зависит от параметров обучения CatBoost (количество итераций, скорость обучения и т. д.).

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor

# 1. Подготовка данных (замените своими данными)
data = {'x1': [1, 2, 3, 4, 5], 'x2': [6, 7, 8, 9, 10], 'y': [11, 12, 13, 14, 15]}
df = pd.DataFrame(data)
X = df[['x1', 'x2']]
y = df['y']

# 2. Создание полиномиальных признаков
poly = PolynomialFeatures(degree=2, include_bias=False)  # degree - степень полинома
X_poly = poly.fit_transform(X)
feature_names = poly.get_feature_names_out(input_features=['x1', 'x2'])  # Получаем имена признаков
X_poly_df = pd.DataFrame(X_poly, columns=feature_names)  # Преобразуем в DataFrame для удобства

# 3. Обучение CatBoost
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=0)
model.fit(X_poly_df, y)

# 4. Получение важности признаков
feature_importance = model.get_feature_importance()

# 5. Создание DataFrame с важностью признаков
importance_df = pd.DataFrame({'Feature': X_poly_df.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values('Importance', ascending=False)

# 6. Отбор наиболее важных признаков (например, топ-N)
top_n = 3  # Выберите желаемое количество
selected_features = importance_df['Feature'][:top_n].tolist()

# 7. Фильтрация данных для использования только отобранных признаков
X_selected = X_poly_df[selected_features]

print("Отобранные признаки:", selected_features)
print(X_selected.head())  # Показываем первые несколько строк с отобранными признаками

**2. SelectFromModel (sklearn):**  
Суть: Использует модель CatBoost в качестве "оценщика" для отбора признаков. Он выбирает признаки на основе заданного порога важности (по умолчанию это среднее значение важности).  
Преимущества: Интегрирован с Scikit-learn, позволяет легко экспериментировать с разными порогами отбора.  
Недостатки: Порог по умолчанию (среднее значение) может не всегда быть оптимальным.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# (Шаги 1-3 как в примере выше)

# 4. Использование SelectFromModel
sfm = SelectFromModel(model, threshold='mean')  # Можно указать конкретный порог вместо 'mean'
sfm.fit(X_poly_df, y)

# 5. Получение отобранных признаков
selected_features = X_poly_df.columns[sfm.get_support()].tolist()

# 6. Фильтрация данных
X_selected = X_poly_df[selected_features]

print("Отобранные признаки:", selected_features)
print(X_selected.head())

**3. Recursive Feature Elimination (RFE):**  
Суть: RFE – это жадный алгоритм, который рекурсивно удаляет признаки и обучает модель на оставшихся признаках. Он оценивает, насколько хорошо модель работает с каждым набором признаков, и выбирает лучший набор.  
Преимущества: Хорошо работает, когда признаки сильно коррелированы. Позволяет напрямую указать желаемое количество признаков.  
Недостатки: Вычислительно дорогой, особенно для больших наборов данных.

In [ ]:
from sklearn.feature_selection import RFE

# (Шаги 1-3 как в примере выше)

# 4. Использование RFE
rfe = RFE(model, n_features_to_select=3)  # Укажите желаемое количество признаков
rfe.fit(X_poly_df, y)

# 5. Получение отобранных признаков
selected_features = X_poly_df.columns[rfe.support_].tolist()

# 6. Фильтрация данных
X_selected = X_poly_df[selected_features]

print("Отобранные признаки:", selected_features)
print(X_selected.head())

**Рекурсивное исключение признаков с кросс-валидацией (RFECV):**

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression

# Создадим синтетический набор данных
X, y = make_regression(n_samples=1000, n_features=20, random_state=42)

# Инициализируем модель
model = LinearRegression()

# Инициализируем RFECV с кросс-валидацией
rfecv = RFECV(estimator=model, step=1, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
rfecv.fit(X, y)

print("Оптимальное количество признаков : %d" % rfecv.n_features_)
print("Выбранные признаки: %s" % rfecv.support_)
print("Рейтинг признаков (1 - лучший): %s" % rfecv.ranking_)

**4. Отбор на основе перестановочной важности (permutation importance):**  
Суть: Перетасовываем значения каждого признака по очереди и смотрим, насколько сильно ухудшается производительность модели. Если ухудшение большое, значит признак важный.  
Преимущества: Позволяет оценить, насколько признаки _действительно_ важны для модели, а не только то, как часто они использовались в процессе обучения.  
Недостатки: Вычислительно более затратный, чем просто получение важности признаков из CatBoost.

In [ ]:
import eli5

# (Шаги 1-3 как в примере выше)

# 4. Вычисление перестановочной важности
permutation_importance = eli5.PermutationImportance(model, random_state=42)
permutation_importance.fit(X_poly_df, y)

# 5. Отображение результатов
eli5.show_weights(permutation_importance, feature_names=X_poly_df.columns.tolist())

# На основе вывода eli5.show_weights() выберите признаки с высокой важностью и отфильтруйте данные
# ...

**5. Отбор признаков на основе перестановок (Permutation Importance):**

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import pandas as pd

# Создадим синтетический набор данных
X, y = make_regression(n_samples=1000, n_features=20, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])  # Для получения названий колонок

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализируем и обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# Вычисляем важность признаков на основе перестановок
result = permutation_importance(model, X_test, y_test, scoring='neg_mean_squared_error', n_repeats=10, random_state=42)

# Сортируем признаки по важности
sorted_idx = result.importances_mean.argsort()[::-1]

print("Важность признаков (перестановки):")
for i in sorted_idx:
    print(f"{X.columns[i]}: {result.importances_mean[i]:.4f} +/- {result.importances_std[i]:.4f}")

# Отбор признаков на основе порога (например, отбираем только положительные значения важности)
selected_features = X.columns[result.importances_mean > 0]
print("\nВыбранные признаки:", selected_features)

**6. Последовательный поиск (Sequential Feature Selection, SFS):**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Создадим синтетический набор данных
X, y = make_regression(n_samples=1000, n_features=20, random_state=42)

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sequential Forward Selection
sfs = SFS(LinearRegression(),
          k_features='best',  # Автоматически выбирает лучшее количество признаков на основе кросс-валидации
          forward=True,
          floating=False,
          scoring='neg_mean_squared_error',
          cv=5)

sfs.fit(X_train, y_train)

print("Последовательный прямой выбор (SFS):")
print("Лучшие признаки:", sfs.k_feature_names_)

**7. Метод отбора признаков на основе генетического алгоритма:**

In [ ]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

# Создадим синтетический набор данных
X, y = make_regression(n_samples=1000, n_features=20, random_state=42)

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализируем TPOTRegressor (настраиваем на отбор признаков с LinearRegression)
tpot = TPOTRegressor(
    generations=5,
    population_size=20,
    offspring_size=None,
    mutation_rate=0.9,
    crossover_rate=0.1,
    scoring='neg_mean_squared_error',
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=-1,
    config_dict={'sklearn.linear_model.LinearRegression': {}},
    template='Selector-Transformer-Regressor',
)

tpot.fit(X_train, y_train)

print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

In [ ]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from catboost import CatBoostRegressor

# Создадим синтетический набор данных
X, y = make_regression(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определяем конфигурацию TPOT, включающую CatBoost
tpot_config = {
        'catboost.CatBoostRegressor': {
        'iterations': [100, 200, 300],  #  Примерные значения, настройте под свою задачу
        'learning_rate': [0.01, 0.03, 0.1], # Примерные значения
        'depth': [4, 6, 8], # Примерные значения
        'l2_leaf_reg': [1, 3, 5], # Примерные значения
        'random_state': [42],
        'verbose': [False]  # Отключаем вывод информации от CatBoost
    },
    # Добавьте другие модели и параметры, если хотите, чтобы TPOT их тоже рассматривал
}


# Инициализируем TPOTRegressor с конфигурацией
tpot = TPOTRegressor(
    generations=5,  #  Увеличьте для более тщательного поиска
    population_size=20, # Увеличьте для более тщательного поиска
    offspring_size=None,
    mutation_rate=0.9,
    crossover_rate=0.1,
    scoring='neg_mean_squared_error',
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=-1,
    config_dict=tpot_config,  # Используем нашу конфигурацию с CatBoost
    template='Selector-Transformer-Regressor', # Убедитесь, что включены этапы отбора признаков
)

tpot.fit(X_train, y_train)

print(tpot.score(X_test, y_test))
tpot.export('tpot_catboost_pipeline.py')

К сожалению, TPOT не предоставляет простого способа напрямую получить список отобранных признаков. Однако вы можете извлечь эту информацию из экспортированного пайплайна (файла .py, созданного с помощью tpot.export()).

**Рекомендации и best practices:**

**•  Кросс-валидация:** Используйте кросс-валидацию при отборе признаков, чтобы убедиться, что выбранные признаки хорошо обобщаются на новых данных. Это очень важно, чтобы избежать переобучения! Например, можно встроить процесс отбора признаков в конвейер (Pipeline) scikit-learn и использовать cross_val_score.

**•  Комбинация методов:** Попробуйте комбинировать разные методы отбора признаков. Например, можно сначала использовать SelectFromModel с мягким порогом, а затем применить RFE для дальнейшего сокращения числа признаков.

**•  Визуализация:** Визуализируйте важность признаков (например, с помощью столбчатой диаграммы), чтобы лучше понять, какие признаки наиболее важны.

**•  Экспериментируйте с параметрами:** Попробуйте разные значения степени полинома (degree), количества итераций CatBoost, порога отбора признаков и т. д.

**•  Ручной отбор:** Иногда полезно провести ручной анализ признаков на основе вашего понимания предметной области. Например, если вы знаете, что определенные комбинации признаков имеют смысл в контексте задачи, вы можете сохранить их, даже если они не очень важны по мнению модели.


**Пример использования Feature Importance с кросс-валидацией:**

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Класс для отбора признаков на основе важности CatBoost
class CatBoostFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, model, top_n=5):
        self.model = model
        self.top_n = top_n
        self.selected_features = None

    def fit(self, X, y=None):
        self.model.fit(X, y)
        feature_importance = self.model.get_feature_importance()
        importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
        importance_df = importance_df.sort_values('Importance', ascending=False)
        self.selected_features = importance_df['Feature'][:self.top_n].tolist()
        return self

    def transform(self, X):
        return X[self.selected_features]


# 1. Подготовка данных (замените своими данными)
data = {'x1': [1, 2, 3, 4, 5], 'x2': [6, 7, 8, 9, 10], 'y': [11, 12, 13, 14, 15]}
df = pd.DataFrame(data)
X = df[['x1', 'x2']]
y = df['y']


# 2. Создание пайплайна
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('selector', CatBoostFeatureSelector(CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=0), top_n=3)),
    ('model', CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=0)) # Обучаем модель после отбора признаков
])


# 3. Кросс-валидация
scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error') # Или другая метрика
print("Кросс-валидация MSE:", -scores.mean()) #  Инвертируем знак, так как cross_val_score максимизирует функцию, а MSE нужно минимизировать

# 4. Обучение пайплайна на всех данных (после кросс-валидации)
pipeline.fit(X, y)

# 5.  Получение отобранных признаков
selected_features = pipeline.named_steps['selector'].selected_features
print("Отобранные признаки:", selected_features)

**PolynomialFeatures**

1. Квадратичные члены для OHE-признаков:

Вы совершенно верно заметили, что возведение в квадрат OHE-признаков не имеет смысла. Поскольку OHE-признаки принимают значения 0 или 1, то 0^2 = 0 и 1^2 = 1. То есть, квадратичные члены не добавят никакой новой информации. Сами по себе квадраты OHE-признаков бесполезны.

2. Произведения OHE-признаков и числового признака:

Произведения между OHE-признаками и числовым признаком (в вашем случае, площади) имеют смысл и часто полезны. Они создают эффект взаимодействия между категориальным и числовым признаками.

•  Интерпретация: Произведение OHE-признака "отделка_улучшенная" и площади можно интерпретировать как "площадь, только для квартир с улучшенной отделкой". По сути, это позволяет модели учитывать разный наклон зависимости целевой переменной от площади для каждой категории отделки. Если коэффициент при этом произведении будет значимым, это говорит о том, что зависимость целевой переменной от площади различна для квартир с улучшенной отделкой по сравнению с другими.

•  Нет проблем с нулями и единицами: Нули и единицы в OHE-признаках в данном случае играют роль "выключателя" и "включателя". Когда OHE-признак равен 0, произведение равно 0, и, следовательно, вклад площади для этой категории отделки не учитывается. Когда OHE-признак равен 1, произведение равно площади, и, следовательно, вклад площади для этой категории отделки учитывается в полном объеме. Это ровно то, что нам нужно для моделирования взаимодействия.

Что делать в вашем случае?

Учитывая все вышесказанное, вот как можно поступить при создании полиномиальных признаков в вашей ситуации:

1. Не создавайте квадратичные члены для OHE-признаков. Просто пропустите их.

2. Создавайте произведения между OHE-признаками и числовым признаком (площадью). Это позволит модели учитывать разный эффект площади для каждой категории отделки.

3. Создавайте квадратичные и кубические члены для площади. Важно помнить про регуляризацию и масштабирование.

4. Подумайте о создании попарных произведений OHE признаков. Их стоит создавать только в том случае, если это имеет смысл с точки зрения предметной области. Попарные произведения создадут новые категориальные признаки, моделирующие совместное присутствие двух категорий. Однако, если ваши категории взаимоисключающие (как в случае с отделкой, где квартира может иметь только один тип отделки), то попарные произведения всегда будут равны нулю.

1. Попарные произведения OHE-признаков одного фактора (состояние отделки):

•   Да, попарные произведения OHE-признаков одного фактора, если фактор взаимоисключающий (как "состояние отделки"), всегда будут равны 0.  Это происходит потому, что в каждой строке данных только один из OHE-признаков может быть равен 1, а все остальные равны 0.  Поэтому, произведение любых двух OHE-признаков из одного и того же фактора всегда будет равно 0.
•   Да, такие признаки (попарные произведения OHE-признаков одного взаимоисключающего фактора) необходимо исключить, так как они не несут никакой полезной информации.  Они только увеличивают размерность пространства признаков и могут запутать модель.

2. Попарные произведения OHE-признаков разных факторов (состояние отделки и местоположение (район)):

•   Да, попарные произведения OHE-признаков разных факторов *допускаются* и часто полезны.  Они моделируют *совместное влияние* или *взаимодействие* этих факторов на целевую переменную.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Пример данных
data = {'отделка': ['стандарт', 'улучшенный', 'без отделки', 'стандарт', 'улучшенный'],
        'площадь': [50, 70, 40, 60, 80],
        'цена': [100, 150, 80, 120, 170]}
df = pd.DataFrame(data)

# 1. OHE для категориального признака
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False для удобства работы
ohe.fit(df[['отделка']])
ohe_features = ohe.transform(df[['отделка']])
ohe_feature_names = ohe.get_feature_names_out(['отделка']) # ['отделка_без отделки', 'отделка_стандарт', 'отделка_улучшенный']
ohe_df = pd.DataFrame(ohe_features, columns=ohe_feature_names)
df = pd.concat([df, ohe_df], axis=1)
df = df.drop('отделка', axis=1) # Удаляем исходный столбец 'отделка'

# 2. Полиномиальные признаки
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False) # interaction_only=False для квадратов чисел

#  Выбираем признаки для полиномиальных преобразований (площадь и OHE признаки)
features_for_poly = ['площадь'] + list(ohe_feature_names)
X = df[features_for_poly]

#  Преобразуем данные, создавая полиномиальные признаки
X_poly = poly.fit_transform(X)

#  Создаем имена для новых признаков.  Это немного сложно, потому что PolynomialFeatures создает много комбинаций
poly_feature_names = poly.get_feature_names_out(features_for_poly)
poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)


#  Удаляем квадраты OHE признаков (они неинформативны, как мы обсуждали)
for feature_name in ohe_feature_names:
    square_feature_name = f"{feature_name}^2"
    if square_feature_name in poly_df.columns:
        poly_df = poly_df.drop(square_feature_name, axis=1)

#  Соединяем полиномиальные признаки с исходными
df = pd.concat([df, poly_df], axis=1)

# Теперь у вас есть DataFrame `df` со всеми нужными признаками
# Добавляем целевую переменную
y = df['цена']
X = df.drop('цена', axis=1)

#  Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и обучаем модель (пример с линейной регрессией и масштабированием)
model = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

model.fit(X_train, y_train)

# Оценка модели
score = model.score(X_test, y_test)
print(f"R^2 score: {score}")

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Пример данных (добавляем признак "район")
data = {'отделка': ['стандарт', 'улучшенный', 'без отделки', 'стандарт', 'улучшенный'],
        'район': ['центр', 'спальный', 'центр', 'спальный', 'центр'],
        'площадь': [50, 70, 40, 60, 80],
        'цена': [100, 150, 80, 120, 170]}
df = pd.DataFrame(data)

# 1. OHE для категориальных признаков
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(df[['отделка', 'район']])
ohe_features = ohe.transform(df[['отделка', 'район']])
ohe_feature_names = ohe.get_feature_names_out(['отделка', 'район'])
ohe_df = pd.DataFrame(ohe_features, columns=ohe_feature_names)
df = pd.concat([df, ohe_df], axis=1)
df = df.drop(['отделка', 'район'], axis=1)

# 2. Создаем имена OHE признаков для "отделки" и "района"
ohe_otdelka_names = [name for name in ohe_feature_names if 'отделка_' in name]
ohe_raion_names = [name for name in ohe_feature_names if 'район_' in name]

# 3. Создаем попарные произведения OHE-признаков разных факторов ("отделка" и "район")
for otdelka_name in ohe_otdelka_names:
    for raion_name in ohe_raion_names:
        df[f'{otdelka_name}_x_{raion_name}'] = df[otdelka_name] * df[raion_name]

# 4. Полиномиальные признаки для площади (квадратичные, кубические и т.д., если нужно)
df['площадь_2'] = df['площадь']**2
# df['площадь_3'] = df['площадь']**3 # и т.д.

#  Удаляем исходные признаки после OHE, если больше не нужны
# X = df.drop('цена', axis=1)

# Создаем и обучаем модель (пример с линейной регрессией и масштабированием)
#  Разделяем на обучающую и тестовую выборки
y = df['цена']
X = df.drop('цена', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

model.fit(X_train, y_train)

# Оценка модели
score = model.score(X_test, y_test)
print(f"R^2 score: {score}")

Этот параметр определяет, нужно ли создавать только взаимодействия между признаками, или также создавать степени отдельных признаков.  
•  interaction_only=True: Если установлено значение True, то будут создаваться только признаки, представляющие собой произведения различных входных признаков. Степени отдельных признаков (например, x1^2, x2^3) создаваться не будут. Это полезно, когда вас интересуют только взаимодействия между признаками, а не их нелинейные преобразования.  
•  interaction_only=False (значение по умолчанию): Если установлено значение False, то будут создаваться как взаимодействия между признаками, так и степени отдельных признаков. То есть, вы получите все возможные полиномиальные комбинации признаков до заданной степени. Это полезно, когда вы хотите, чтобы модель могла учитывать как взаимодействия, так и нелинейные эффекты отдельных признаков.

Этот параметр определяет, нужно ли включать в выходные данные столбец "смещения" (bias, intercept).  
•  include_bias=True (значение по умолчанию, если не указано другое): Если установлено значение True, то в выходные данные будет добавлен столбец, состоящий из одних единиц. Этот столбец представляет собой "фиктивный" признак, который позволяет модели иметь свободный член (intercept) в линейной комбинации признаков. Многие модели (например, LinearRegression) автоматически добавляют свободный член, поэтому этот столбец часто не нужен и может быть даже вреден (например, вызвать мультиколлинеарность).  
•  include_bias=False: Если установлено значение False, то столбец смещения добавляться не будет. Это полезно, если вы уверены, что ваша модель уже учитывает смещение (например, если вы используете модель, которая автоматически добавляет свободный член, или если вы явно добавляете столбец с единицами в свои данные).